In [9]:
import pandas as pd
import os
import json
import datetime as dt
import math

In [10]:
def read_json(filepath):
    f = open(filepath)
    data = json.load(f)
    return pd.DataFrame.from_dict(data)

In [11]:
team_df = read_json("./esports-data/teams.json")
mapping_df = read_json("./esports-data/mapping_data.json")
game_df = read_json("./games/LPL_A_343996.json")

In [12]:
set(game_df["eventType"].values)

{'building_destroyed',
 'champion_kill',
 'champion_kill_special',
 'champion_level_up',
 'epic_monster_kill',
 'epic_monster_spawn',
 'game_end',
 'game_info',
 'item_destroyed',
 'item_purchased',
 'item_sold',
 'item_undo',
 'objective_bounty_prestart',
 'queued_dragon_info',
 'queued_epic_monster_info',
 'skill_level_up',
 'stats_update',
 'summoner_spell_used',
 'turret_plate_destroyed',
 'turret_plate_gold_earned',
 'ward_killed',
 'ward_placed'}

In [81]:
pd.set_option('display.max_columns', None)
sum(game_df[(game_df["eventType"] == "turret_plate_gold_earned")]["bounty"])

2272.0

In [14]:
# real full game time is 23:35
# https://www.youtube.com/watch?v=37JuelCihvM

# game_info -> game_end
end_time_str =  game_df[game_df["eventType"] == "game_end"]["eventTime"].values[0]
end_time = dt.datetime.strptime(end_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")

In [15]:
# first stats_update to game_end

new_start_time_str =  game_df[game_df["eventType"] == "stats_update"]["eventTime"].values[0]
new_start_time = dt.datetime.strptime(new_start_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
print(end_time - new_start_time)

0:24:25.564000


In [22]:
def split_by_time(dataframe, early_game = 14, late_game = 25):
    new_df = dataframe.copy()
    new_df["dateTime"] = pd.to_datetime(new_df["eventTime"]).dt.tz_localize(None)
    start_time_str = new_df[new_df["eventType"] == "game_info"]["eventTime"].values[0]
    start_time = dt.datetime.strptime(start_time_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    early_game_cutoff = start_time + dt.timedelta(minutes=early_game)
    late_game_cutoff = start_time + dt.timedelta(minutes=late_game)
    early_game_df = new_df[new_df["dateTime"] <= early_game_cutoff]
    mid_game_df = new_df[(new_df["dateTime"] >= early_game_cutoff) & (new_df["dateTime"] <= late_game_cutoff)]
    late_game_df = new_df[new_df["dateTime"] >= late_game_cutoff]
    return (early_game_df, mid_game_df, late_game_df)

In [24]:
game_splits = split_by_time(game_df)
early_game_df = game_splits[0]

In [86]:
game_id = game_df["platformGameId"].values[0]

game_end = game_df[game_df["eventType"] == "game_end"]
winning_team = int(game_end["winningTeam"].values[0])

team_id = mapping_df[mapping_df["platformGameId"] == game_id]["teamMapping"].values[0][str(winning_team)]

team_name = team_df[team_df["team_id"] == team_id]["name"].values[0]
first_blood = int(winning_team == math.ceil(game_df[game_df["killType"] == "firstBlood"]["killer"] / 5) * 100)

early_kill_gained = len(early_game_df[(early_game_df["eventType"] == "champion_kill") & (early_game_df["killer"] <= 5)])
early_kill_given = len(early_game_df[(early_game_df["eventType"] == "champion_kill") & (early_game_df["killer"] > 5)])

early_tower_taken = len(early_game_df[(early_game_df["eventType"] == "building_destroyed") & (early_game_df["teamID"] == winning_team)])
early_tower_given = len(early_game_df[(early_game_df["eventType"] == "building_destroyed") & (early_game_df["teamID"] != winning_team)])

early_dragon_taken = len(early_game_df[(early_game_df["killerTeamID"] == winning_team) & (early_game_df["monsterType"] == "dragon")])
early_dragon_given = len(early_game_df[(early_game_df["eventType"] == "epic_monster_kill") & (early_game_df["killerTeamID"]!= winning_team) & (early_game_df["monsterType"] == "riftHerald")])

early_rift_taken = len(early_game_df[(early_game_df["killerTeamID"] == winning_team) & (early_game_df["monsterType"] == "riftHerald")])
early_rift_given = len(early_game_df[(early_game_df["eventType"] == "epic_monster_kill") & (early_game_df["killerTeamID"]!= winning_team) & (early_game_df["monsterType"] == "riftHerald")])

early_plate_gold_taken = sum(early_game_df[(early_game_df["eventType"] == "turret_plate_gold_earned") & (early_game_df["teamID"] == winning_team)]["bounty"])
early_plate_gold_given = sum(early_game_df[(early_game_df["eventType"] == "turret_plate_gold_earned") & (early_game_df["teamID"] != winning_team)]["bounty"])

features = {"game_id": [game_id], "winning_team": [winning_team], "team_id": [team_id], "team_name": [team_name], "first_blood": [first_blood], "early_kill_gained": [early_kill_gained],
            "early_kill_given": [early_kill_given], "early_tower_taken": [early_tower_taken], "early_tower_given": [early_tower_given], "early_dragon_taken": [early_dragon_taken],
            "early_dragon_given": early_tower_given, "early_rift_taken": early_rift_taken, "early_rift_given": early_rift_given}

C:\Users\Brian\AppData\Local\Temp\ipykernel_8464\1803495779.py:9: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  first_blood = int(winning_team == math.ceil(game_df[game_df["killType"] == "firstBlood"]["killer"] / 5) * 100)


In [87]:
early_plate_gold_taken, early_plate_gold_given

(874.0, 873.0)

In [78]:
features_df = pd.DataFrame()
features_df = pd.concat([features_df, pd.DataFrame.from_dict(features)], ignore_index=True)

features_df

,game_id,winning_team,team_id,team_name,first_blood,early_kill_gained,early_kill_given,early_tower_taken,early_tower_given,early_dragon_taken,early_dragon_given,early_rift_taken,early_rift_given
0,LPL_A:343996,100,99566404853058754,WeiboGaming FAW AUDI,1,6,4,0,0,1,0,0,1
